In [10]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import load_boston
#from sklearn.model_selection import cross_val_predict, cross_validate, cross_val_score, train_test_split, KFold
from sklearn.model_selection import *
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from itertools import combinations
import recordlinkage
from recordlinkage.datasets import load_febrl4
%matplotlib inline

# Project 1

## Problem statement:
Identify duplicated customers/accounts among multiple data sources. For example we have same customer but under different names in SalesForce, Finance system, CRM system.It is very difficult to get a meaningful analysis for our sales team, finance team, marketing team.
    
    
## Data
This is a common issues with medical care for the healthy department and government as well. I will use the data publised by Freely Extensible Biomedical Record Linkage (FEBRL) project.
      
http://recordlinkage.readthedocs.io/en/latest/ref-datasets.html

http://users.cecs.anu.edu.au/~Peter.Christen/Febrl/febrl-0.3/febrldoc-0.3/manual.html  
    
    
    
## Hypothesis
There are duplicated records in the data sources. The expected result would be any duplicated record by removed or linked.
    
## Potential API or Lib can help

- sklearn
- Record linkage toolkit (used this before)
- Levenshtin distance (used fuzzyWuzzy before)
- Builclassifier dedup lib
- Nazca lib




In [11]:
# EDA
# http://recordlinkage.readthedocs.io/en/latest/ref-datasets.html
"""load_febrl4() generated as one data set with 10000 records (5000 originals and 5000 duplicates, with one duplicate per original),
#the originals have been split from the duplicates, into dataset4a.csv (containing the 5000 original records) and 
dataset4b.csv (containing the 5000 duplicate records) These two data sets can be used for testing linkage procedures."""
dfA, dfB = load_febrl4()
dfA.head(2)

,given_name,surname,street_number,address_1,address_2,suburb,postcode,state,date_of_birth,soc_sec_id
rec_id,,,,,,,,,,
rec-1070-org,michaela,neumann,8,stanley street,miami,winston hills,4223,nsw,19151111,5304218
rec-1016-org,courtney,painter,12,pinkerton circuit,bega flats,richlands,4560,vic,19161214,4066625


In [12]:
dfB.head(2)

,given_name,surname,street_number,address_1,address_2,suburb,postcode,state,date_of_birth,soc_sec_id
rec_id,,,,,,,,,,
rec-561-dup-0,elton,NaN,3,light setreet,pinehill,windermere,3212,vic,19651013,1551941
rec-2642-dup-0,mitchell,maxon,47,edkins street,lochaoair,north ryde,3355,nsw,19390212,8859999


# Project 2

## Problem statement:
Identify whether a potential customer will churn when their current contract is ended. It is more expensive to acquire a new customer than to keep existing one from leaving.I am hoping to use this model to build useful tool to predict customer life time value, help sales team to allocate resource to prevent customer leaving.
    
    
    
## Data
Using the telecom churn data provided by kaggle
    
https://www.kaggle.com/becksddf/churn-in-telecoms-dataset/version/1.
    
https://bigml.com/user/francisco/gallery/dataset/5163ad540c0b5e5b22000383
    
    
## Hypothesis
Predict whether a given customer will churn or retain based on all the features.
    
## Potential API or Lib can help
- sklearn
- ...
- Telecom Churn Notebook https://www.kaggle.com/arpitharavi/telecom-churn-notebook





In [13]:
#EDA
tele = pd.read_csv('./data/bigml.csv')
tele.head(2)


,state,account length,area code,phone number,international plan,voice mail plan,number vmail messages,total day minutes,total day calls,total day charge,...,total eve calls,total eve charge,total night minutes,total night calls,total night charge,total intl minutes,total intl calls,total intl charge,customer service calls,churn
0,KS,128,415,382-4657,no,yes,25,265.1,110,45.07,...,99,16.78,244.7,91,11.01,10.0,3,2.7,1,False
1,OH,107,415,371-7191,no,yes,26,161.6,123,27.47,...,103,16.62,254.4,103,11.45,13.7,3,3.7,1,False


In [34]:
print("The features are:")
[x for x in tele.columns[:-1]]


The features are:


['state',
 'account length',
 'area code',
 'phone number',
 'international plan',
 'voice mail plan',
 'number vmail messages',
 'total day minutes',
 'total day calls',
 'total day charge',
 'total eve minutes',
 'total eve calls',
 'total eve charge',
 'total night minutes',
 'total night calls',
 'total night charge',
 'total intl minutes',
 'total intl calls',
 'total intl charge',
 'customer service calls']

In [38]:
print("The target is {}".format(tele.columns[-1]))

The target is churn


In [15]:
tele.describe()

,account length,area code,number vmail messages,total day minutes,total day calls,total day charge,total eve minutes,total eve calls,total eve charge,total night minutes,total night calls,total night charge,total intl minutes,total intl calls,total intl charge,customer service calls
count,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000
mean,101.064806,437.182418,8.099010,179.775098,100.435644,30.562307,200.980348,100.114311,17.083540,200.872037,100.107711,9.039325,10.237294,4.479448,2.764581,1.562856
std,39.822106,42.371290,13.688365,54.467389,20.069084,9.259435,50.713844,19.922625,4.310668,50.573847,19.568609,2.275873,2.791840,2.461214,0.753773,1.315491
min,1.000000,408.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,23.200000,33.000000,1.040000,0.000000,0.000000,0.000000,0.000000
25%,74.000000,408.000000,0.000000,143.700000,87.000000,24.430000,166.600000,87.000000,14.160000,167.000000,87.000000,7.520000,8.500000,3.000000,2.300000,1.000000
50%,101.000000,415.000000,0.000000,179.400000,101.000000,30.500000,201.400000,100.000000,17.120000,201.200000,100.000000,9.050000,10.300000,4.000000,2.780000,1.000000
75%,127.000000,510.000000,20.000000,216.400000,114.000000,36.790000,235.300000,114.000000,20.000000,235.300000,113.000000,10.590000,12.100000,6.000000,3.270000,2.000000
max,243.000000,510.000000,51.000000,350.800000,165.000000,59.640000,363.700000,170.000000,30.910000,395.000000,175.000000,17.770000,20.000000,20.000000,5.400000,9.000000
